<a href="https://colab.research.google.com/github/paoaleman19/M22-Home_sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,014 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,803 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updat

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [13]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",",header=True)

In [19]:
#getting only the year from the "date"column in the data frame
from pyspark.sql.functions import year
home_sales = home_sales.withColumn("year", year(home_sales["date"]))
home_sales.show(15)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [20]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView("home_sales_df")


In [25]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT year, ROUND(AVG(price), 2)
FROM home_sales_df
WHERE bedrooms = 4
GROUP BY year
ORDER BY year ASC
""").show()


+----+--------------------+
|year|round(avg(price), 2)|
+----+--------------------+
|2019|            300263.7|
|2020|           298353.78|
|2021|           301819.44|
|2022|           296363.88|
+----+--------------------+



In [26]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price), 2)
FROM home_sales_df
WHERE (bedrooms = 3 AND bathrooms = 3)
GROUP BY date_built
ORDER BY date_built ASC
""").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [27]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price), 2)
FROM home_sales_df
WHERE (bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000)
GROUP BY date_built
ORDER BY date_built ASC
""").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+



In [36]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2)
FROM home_sales_df
WHERE price >= 350000
GROUP BY view
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  16|           399586.53|
+----+--------------------+
only showing top 20 rows

--- 0.6088452339172363 seconds ---


In [37]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales_df")

DataFrame[]

In [38]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_df')

True

In [41]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2)
FROM home_sales_df
WHERE price >= 350000
GROUP BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  16|           399586.53|
+----+--------------------+
only showing top 20 rows

--- 0.3533143997192383 seconds ---


In [46]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("partitioned_by_dateBuilt")

In [47]:
# 11. Read the parquet formatted data.
partitioned_by_DateBuilt_df =spark.read.parquet("partitioned_by_dateBuilt")

In [48]:
# 12. Create a temporary table for the parquet data.
partitioned_by_DateBuilt_df.createOrReplaceTempView("part_home_sales")

In [49]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2)
FROM part_home_sales
WHERE price >= 350000
GROUP BY view
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           403005.77|
|  51|           788128.21|
|  15|            404673.3|
|  54|           798684.82|
|  11|           399548.12|
|  29|           397771.65|
|  69|           750537.94|
|  42|            396964.5|
|  73|           752861.18|
|  87|           1072285.2|
|  64|           767036.67|
|   3|            398867.6|
|  30|            397862.0|
|  34|           401419.75|
|  59|            791453.0|
|   8|           398592.71|
|  28|           402124.62|
|  22|           402022.68|
|  85|          1056336.74|
|  35|           401934.21|
+----+--------------------+
only showing top 20 rows

--- 0.9301345348358154 seconds ---


In [50]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table part_home_sales")

DataFrame[]

In [51]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("part_home_sales")


False